In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, concat_ws, initcap, when, monotonically_increasing_id, split, explode,
    trim, regexp_replace, row_number, upper, translate, rand, floor, lit
)
from pyspark.sql.types import IntegerType
from pyspark.sql.window import Window

# =======================================================================
# 1. CONFIGURAÇÃO E INICIALIZAÇÃO DO SPARK
# =======================================================================
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.11.901')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')

spark = SparkSession.builder.config(conf=conf).getOrCreate()

# Função para remover acentos de uma coluna
def remover_acentos(df, nome_coluna):
    acentos = "áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ"
    sem_acentos = "aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC"
    return df.withColumn(nome_coluna, translate(col(nome_coluna), acentos, sem_acentos))

# =======================================================================
# 2. LISTA DE BAIRROS (SEM ACENTO E EM MAIÚSCULO)
# =======================================================================
bairros_sp = {
    "CENTRO": [
        "SE", "BELA VISTA", "BOM RETIRO", "CAMBUCI", "CONSOLACAO",
        "LIBERDADE", "REPUBLICA", "SANTA CECILIA"
    ],
    "NORTE": [
        "CASA VERDE", "CACHOEIRINHA", "LIMAO", "BRASILANDIA",
        "FREGUESIA DO O", "JACANA", "TREMEMBE", "PERUS",
        "ANHANGUERA", "PIRITUBA", "JARAGUA", "SAO DOMINGOS",
        "SANTANA", "TUCURUVI", "MANDAQUI", "VILA MARIA", "VILA GUILHERME"
    ],
    "SUL": [
        "CAMPO BELO", "CAMPO LIMPO", "CAPAO REDONDO", "CIDADE ADEMAR",
        "CIDADE DUTRA", "CURSINO", "GRAJAU", "ITAIM BIBI",
        "IPIRANGA", "JABAQUARA", "JARDIM ANGELA", "JARDIM SAO LUIS",
        "MARSILAC", "MOEMA", "MORUMBI", "PARELHEIROS", "PEDREIRA",
        "SACOMA", "SANTO AMARO", "SOCORRO", "SAUDE", "VILA ANDRADE",
        "VILA MARIANA", "VILA OLIMPIA"
    ],
    "LESTE": [
        "AGUA RASA", "ARICANDUVA", "ARTUR ALVIM", "BELEM", "BRAS",
        "CANGAIBA", "CARRAO", "CIDADE LIDER", "CIDADE TIRADENTES",
        "ERMELINO MATARAZZO", "GUAJANASES", "ITAIM PAULISTA",
        "ITAQUERA", "JARDIM HELENA", "JOSE BONIFACIO", "LAJEADO",
        "MOOCA", "PARI", "PARQUE DO CARMO", "PENHA", "PONTE RASA",
        "SAO LUCAS", "SAO MATEUS", "SAO MIGUEL", "SAO RAFAEL",
        "SAPOPEMBA", "TATUAPE", "VILA CURUCA", "VILA FORMOSA",
        "VILA JACUI", "VILA MATILDE", "VILA PRUDENTE"
    ],
    "OESTE": [
        "ALTO DE PINHEIROS", "BARRA FUNDA", "BUTANTA", "JAGUARA",
        "JAGUARE", "JARDIM PAULISTA", "JARDIM PAULISTANO", "JARDIM EUROPA",
        "JARDIM AMERICA", "LAPA", "MORUMBI", "PERDIZES", "PINHEIROS",
        "RAPOSO TAVARES", "RIO PEQUENO", "VILA LEOPOLDINA", "VILA MADALENA",
        "VILA SONIA", "ITAIM BIBI"
    ]
}

# Flatten lista de bairros
lista_bairros = [b for bairros in bairros_sp.values() for b in bairros]

# =======================================================================
# 3. LEITURA E LIMPEZA INICIAL DOS DADOS
# =======================================================================
df_raw = spark.read.option('delimiter', ';') \
                     .option('header', 'true') \
                     .option('nullValue', 'null') \
                     .option('encoding', 'UTF-8') \
                     .csv('s3a://bucket-raw-upa-connect-otavio/part-00000-20b337c6-bd01-406a-912f-15a1d9129c22.c000.csv')

df_selecionado = df_raw.select("idade", "sexo", "sintomas", "municipio", "outrosSintomas")

df_selecionado = remover_acentos(df_selecionado, "municipio")
df_selecionado = remover_acentos(df_selecionado, "sexo")
df_selecionado = remover_acentos(df_selecionado, "sintomas")
df_selecionado = remover_acentos(df_selecionado, "outrosSintomas")
df_selecionado = df_selecionado.withColumn("municipio", upper(col("municipio")))

df_limpo = df_selecionado.filter(col('idade').isNotNull() & (col('idade') != ''))
df_limpo = df_limpo.withColumn("idade", col("idade").cast(IntegerType()))

# =======================================================================
# 4. TRATAMENTO DE SINTOMAS
# =======================================================================
df_sintomas_unificados = df_limpo.withColumn(
    "sintomas",
    initcap(
        when(
            col("outrosSintomas").isNull() | (col("outrosSintomas") == ''),
            col("sintomas")
        ).otherwise(
            concat_ws(", ", col("sintomas"), col("outrosSintomas"))
        )
    )
).drop("outrosSintomas")

# =======================================================================
# 5. SUBSTITUIR MUNICIPIO POR BAIRRO ALEATÓRIO (COM REPOSIÇÃO)
# =======================================================================
df_sao_paulo = df_sintomas_unificados.filter(col('municipio') == 'SAO PAULO')

# número total de bairros
n_bairros = len(lista_bairros)

# índice aleatório para cada linha (reprodutível com seed)
df_sao_paulo = df_sao_paulo.withColumn("idx_bairro", floor(rand(seed=42) * n_bairros))

# cria coluna bairro a partir da lista
mapping_expr = when(lit(1) == 0, lit(None))  # dummy inicial
for i, bairro in enumerate(lista_bairros):
    mapping_expr = mapping_expr.when(col("idx_bairro") == i, lit(bairro))

df_com_bairro = df_sao_paulo.withColumn("bairro", mapping_expr).drop("municipio", "idx_bairro")

# =======================================================================
# 6. GERAÇÃO DE ID
# =======================================================================
window_spec = Window.orderBy(monotonically_increasing_id())
df_com_id = df_com_bairro.withColumn("id", row_number().over(window_spec))

# =======================================================================
# 7. EXPLOSÃO E LIMPEZA FINAL DOS SINTOMAS
# =======================================================================
# Substitui ' e ' e '/' por vírgulas para padronizar os separadores de sintomas.
df_sintomas_separados = df_com_id.withColumn(
    "sintomas", regexp_replace(col("sintomas"), " E ", ", ")
).withColumn(
    "sintomas", regexp_replace(col("sintomas"), "/", ", ")
)

# Substitui o sinal de '+' e remove qualquer vírgula duplicada para garantir que a separação
# posterior funcione corretamente. Também remove o ponto final no final da string, se existir.
df_sintomas_separados = df_sintomas_separados.withColumn(
    "sintomas", regexp_replace(col("sintomas"), "\\+", ", ")
).withColumn(
    "sintomas", regexp_replace(col("sintomas"), ",,", ",")
).withColumn(
    "sintomas", regexp_replace(col("sintomas"), "\\.$", "")
)

# A função `split` divide a string de sintomas em um array de strings usando a vírgula como delimitador.
# Em seguida, `explode` cria uma nova linha para cada sintoma no array, "explodindo" a lista de sintomas
# em linhas individuais.
df_explodido = df_sintomas_separados.withColumn(
    "sintoma_individual", explode(split(col("sintomas"), ", "))
)

# Remove espaços em branco do início e do fim de cada sintoma individual para limpeza.
# Filtra linhas onde o sintoma individual é uma string vazia.
df_normalizado = df_explodido.withColumn(
    "sintoma_individual", trim(col("sintoma_individual"))
).filter(col("sintoma_individual") != '')

# Filtra linhas que contêm números, removendo sintomas que são, na verdade, códigos ou outras informações numéricas.
df_normalizado = df_normalizado.filter(~col("sintoma_individual").rlike("[0-9]"))

# Remove o acento agudo, se presente, para padronizar os sintomas.
df_normalizado = df_normalizado.withColumn(
    "sintoma_individual",
    regexp_replace(col("sintoma_individual"), "´", "")
)

# Substitui a string "Outros" por uma string vazia e depois filtra, removendo-a completamente dos dados.
df_normalizado = df_normalizado.withColumn(
    "sintoma_individual",
    when(col("sintoma_individual") == "Outros", "").otherwise(col("sintoma_individual"))
)

df_normalizado = df_normalizado.filter(col("sintoma_individual") != "")

# =======================================================================
# 8. SELEÇÃO FINAL
# =======================================================================
df_final = df_normalizado.select(
    "id",
    "idade",
    "sexo",
    upper(col("sintoma_individual")).alias("sintoma_individual"),
    "bairro"
)

print("Amostra dos dados tratados:")
df_final.show(n=100, truncate=False)

# =======================================================================
# 9. ANÁLISE DE FREQUÊNCIA
# =======================================================================
df_contagem_sintomas = df_final.groupBy("sintoma_individual").count().sort(col("count").desc())
print("\nContagem de frequência de cada sintoma:")
df_contagem_sintomas.show(n=100, truncate=False)


:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1d0cbe3e-8de9-4eb7-bce8-f3b4a06ad207;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 528ms :: artifacts dl 19ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 by [com.amazonaws#aws-java-sdk-bundle;1.12.262] in [default]
	---------------------------------------------------------------------
	|     

Amostra dos dados tratados:


25/09/09 23:43:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/09 23:43:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/09 23:43:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/09 23:44:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/09 23:44:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---+-----+---------+------------------+-----------------+
|id |idade|sexo     |sintoma_individual|bairro           |
+---+-----+---------+------------------+-----------------+
|1  |14   |Feminino |CORIZA            |ITAQUERA         |
|1  |14   |Feminino |TOSSE             |ITAQUERA         |
|2  |1    |Feminino |CORIZA            |ARICANDUVA       |
|2  |1    |Feminino |DOR DE GARGANTA   |ARICANDUVA       |
|2  |1    |Feminino |FEBRE             |ARICANDUVA       |
|3  |49   |Feminino |TOSSE             |BUTANTA          |
|3  |49   |Feminino |DOR DE GARGANTA   |BUTANTA          |
|3  |49   |Feminino |MAL ESTAR NO CORPO|BUTANTA          |
|4  |4    |Masculino|CORIZA            |CAMPO LIMPO      |
|4  |4    |Masculino|TOSSE             |CAMPO LIMPO      |
|4  |4    |Masculino|FEBRE             |CAMPO LIMPO      |
|5  |19   |Feminino |SINDROME GRIPAL   |PARQUE DO CARMO  |
|6  |2    |Feminino |CORIZA            |ARTUR ALVIM      |
|6  |2    |Feminino |TOSSE             |ARTUR ALVIM     

+------------------------+------+
|sintoma_individual      |count |
+------------------------+------+
|TOSSE                   |205524|
|CORIZA                  |166975|
|DOR DE CABECA           |145440|
|FEBRE                   |141098|
|DOR DE GARGANTA         |95852 |
|ASSINTOMATICO           |39154 |
|DISPNEIA                |32586 |
|MIALGIA                 |12917 |
|DOR NO CORPO            |12006 |
|DISTURBIOS GUSTATIVOS   |11369 |
|DISTURBIOS OLFATIVOS    |10042 |
|DIARREIA                |4095  |
|CONGESTAO NASAL         |3375  |
|VOMITO                  |2495  |
|SINTOMAS GRIPAIS        |2392  |
|SINDROME GRIPAL         |1873  |
|CANSACO                 |1734  |
|FRAQUEZA                |1362  |
|MAL ESTAR               |1204  |
|NAUSEAS                 |1184  |
|CALAFRIOS               |1113  |
|FADIGA                  |1031  |
|DOR NAS COSTAS          |999   |
|ODINOFAGIA              |955   |
|ESPIRROS                |938   |
|NAUSEA                  |935   |
|DOR ABDOMINAL

In [2]:
# =======================================================================
# 8. SALVANDO O RESULTADO NO S3
# =======================================================================
# Define o caminho de saída no S3
output_path = "s3a://bucket-trusted-upa-connect-otavio/tratamento_final_gripe"

df_final.coalesce(1).write.option("header", "true").option("delimiter", ";").mode("overwrite").csv(output_path)

print(f"\nDados salvos com sucesso em: {output_path}")

25/09/09 23:44:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/09 23:44:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/09 23:44:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/09 23:45:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/09 23:45:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/09 23:45:09 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
25/09/09 23:45:10 WARN AbstractS3AC


Dados salvos com sucesso em: s3a://bucket-trusted-upa-connect-otavio/tratamento_final_gripe
